# **After cleaning**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':18})

In [2]:
import pyspark as ps
import pandas as pd
from pyspark.sql.functions import lit

In [10]:
# look at df created with spark importer

df = pd.read_csv('../data/2020-08-11_all_clean.csv', lineterminator='\n', parse_dates=['tweet_date_created', 'user_date_created'])
df.head()

,tweet_id,state,search_term_key,tweet_date_created,user_id,user_date_created,location,description,user_verified,tweet_text_wo_mentions,source_text
0,1246830993903955968,AR,@joebiden,2020-04-05 16:04:08+00:00,29214401,2009-04-06 14:53:51+00:00,NaN,The son of Rage and Love,False,"lil pump really said ""look at my wrist corona ...",Twitter for Android
1,1246830994537222144,AR,@joebiden,2020-04-05 16:04:08+00:00,4766479674,2016-01-16 04:42:14+00:00,"geohell, disneyworld",i'm a fully automated toilet,False,@Stephan59177054 @NomikiKonst @JoeBiden I thou...,Twitter for Android
2,1246830995132813312,AR,@joebiden,2020-04-05 16:04:08+00:00,3166145611,2015-04-21 02:02:20+00:00,NaN,Trump supporter! I follow back all Patriots. ...,False,RT @Jali_Cat: Yet you’ve been in office since ...,Twitter Web App
3,1246830995761963008,AR,@joebiden,2020-04-05 16:04:09+00:00,2868989345,2014-11-09 14:12:09+00:00,"North Huntingdon, PA",NaN,False,@JoeBiden This is a better message and more co...,Twitter for iPad
4,1246830995795369984,AR,@joebiden,2020-04-05 16:04:09+00:00,91865200,2009-11-22 20:47:43+00:00,Colorado,#the Resistance🇱🇷,False,RT @KaivanShroff: “I think it’s close to crimi...,Twitter for iPhone


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117652 entries, 0 to 117651
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype              
---  ------                  --------------   -----              
 0   tweet_id                117652 non-null  int64              
 1   state                   117652 non-null  object             
 2   search_term_key         117652 non-null  object             
 3   tweet_date_created      117652 non-null  datetime64[ns, UTC]
 4   user_id                 117652 non-null  int64              
 5   user_date_created       117652 non-null  datetime64[ns, UTC]
 6   location                85966 non-null   object             
 7   description             99927 non-null   object             
 8   user_verified           117652 non-null  bool               
 9   tweet_text_wo_mentions  117652 non-null  object             
 10  source_text             117652 non-null  object             
dtypes: bool(1), datetime64[ns,

In [13]:
acount_age = df['tweet_date_created'] - df['user_date_created']

In [17]:
acount_age[0].

AttributeError: 'Timedelta' object has no attribute 'day'